In [1]:
import sys
import os

#Very important to use the .venv as the kernel of the project zhs - (source my_project_path/.venv/bin/activate)

def set_project_path_and_virtualenv():
    
    # Get the current working directory
    current_path = os.getcwd()

    # Extract the project path from the current working directory
    project_path = os.path.dirname(current_path)

    if current_path in sys.path:
        sys.path.remove(current_path)

    # Append project path to sys.path
    sys.path.append(project_path)



    # The next steps are not needed as long as the kernel is initialized in the - venv

    venv_name = '.venv'

    # # Construct the path to the activate script in the virtual environment
    activate_script = os.path.join(project_path, venv_name, 'bin', 'activate')

    # # Set the PATH environment variable to only include the virtual environment
    os.environ['PATH'] = os.path.dirname(activate_script)

    # env_path = os.path.join(project_path, venv_name, 'lib', 'python3.11' , 'site-packages')
    # sys.path.append(env_path)

# Call the function
set_project_path_and_virtualenv()

In [2]:
import pandas as pd
from IPython.display import display
from io import StringIO
import ee
# pip install earthengine-api
import requests
import time


import src.stages.image_mining as mining_package

n_images_class = 1000
# List of levels to keep
levels_to_merge = []

#     '52.a.05.006+52.b.08.101',
#      '52.a.05.006+51.a.01.101',
    # '52.a.05.001+52.b.09.101',
    # '61.a.02.004+55.a.02.101',
    # '61.a.04.010+55.a.02.101',
    # '75.a.02.012A+65.a.04.006+50.a.01.101+50.a.03.101+57.a.03.101',
    # '50.a.01.101+50.a.03.101',
    # '59.b.04.101+59.b.06.101',
    # '52.b.08.101+27.a.03.101',
    # '52.b.09.101',
    # '76.c.10.001+64.a.05.003+50.c.13.101',
    # '74.a.02.001+52.b.07.101+64.a.05.017+50.c.13.101+56.a.05.101'


/var/folders/vz/gbtwmbg50fj0wjbh42bpfh1m0000gn/T/ipykernel_51242/4104947018.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


### **Grouping Classes Based on Requirements**

In [3]:
df_load = mining_package.DataLoader("Sampled_CoordinatesM4(2).csv")
df = df_load.read_csv()

df_customize = mining_package.PreprocessClasses(levels_to_merge,n_images_class)
df = df_customize.clean_and_filter(df)

df_final = mining_package.ImageNameProcessor()
df= df_final.process_image_names(df)

Number of ORIGINAL unique levels in the 'CVB1': 1
Number of CURRENT unique levels in the 'CVB1': 1
Unique original values in the 'CVB1': ['61.a.02.004']
Unique current values in the 'CVB1': ['61.a.02.004']


### **Visualizamos df**

In [4]:
df.head(2)

,rand_point_id,fid,COD_TESELA,SITE_CODE,COD_TV,NOMBRE,CVB1,CVB2,CVB3,CVB4,CVB5,Area,long,lat,Image_Name,Image_Name_Length
0,0,94052,29TPG88-13-209015,ES4190110,61.a.02.004+55.a.02.101,Brezales con Erica australis con pastos vivace...,61.a.02.004,55.a.02.101,NaN,NaN,NaN,167171,-6.758186,42.267494,94052:61.a.02.004,17
1,671,133599,29TPH95-10-154057,ES4130010,61.a.02.004+55.a.02.101,Brezales con Erica australis con pastos vivace...,61.a.02.004,55.a.02.101,NaN,NaN,NaN,348144,-6.577618,42.895564,133599:61.a.02.004,17


### **Autentificación Earth Engine**

In [5]:
ee.Authenticate()
ee.Initialize()

### **Función de obtención de imagenes**

##### Importante preparar las carpetas con sus nombres en google drive!

In [6]:


# help(mining_package.EE_Requests)

run_mining = mining_package.EE_Requests()

run = run_mining.Init_EE_Request(df,'COPERNICUS/S2_SR_HARMONIZED', '2023-07-01','2023-09-01','model4',150)

print("TOTAL Execution time:", run['execution_time'], "min")
print("ERRORS:", run['count_error'])
print("SUCCESS:", run['count_success'])
print("ERROR LIST:", run['list_error'])
print("SUCCESS LIST:", run['list_success'])


Iteration number:  1
Completed: 94052:61.a.02.004
Execution time completion:  0.3695162057876587 min
Iteration number:  2
Completed: 133599:61.a.02.004
Execution time completion:  0.19078410069147747 min
Iteration number:  3
Completed: 133547:61.a.02.004
Execution time completion:  0.2713098128636678 min
Iteration number:  4
Completed: 133547:61.a.02.004
Execution time completion:  0.27406861782073977 min
Iteration number:  5
Completed: 133547:61.a.02.004
Execution time completion:  0.2772987206776937 min
Iteration number:  6
Completed: 133547:61.a.02.004
Execution time completion:  0.2744607170422872 min
Iteration number:  7
Completed: 133547:61.a.02.004
Execution time completion:  0.36587812900543215 min
Iteration number:  8
Completed: 133547:61.a.02.004
Execution time completion:  0.4528871973355611 min
Iteration number:  9
Completed: 133547:61.a.02.004
Execution time completion:  0.27498438358306887 min
Iteration number:  10
Completed: 133547:61.a.02.004
Execution time completion: 

##Nota sobre COPERNICUS HARMONIZED

COPERNICUS/S2:
The "COPERNICUS/S2" dataset in Earth Engine contains raw Sentinel-2 Level-1C (L1C) data. Level-1C data refers to the original data acquired by the satellite, which undergoes minimal processing and is provided in its original geometry and radiometric format. The imagery in this dataset includes multiple spectral bands at different spatial resolutions.

COPERNICUS/S2_SR_HARMONIZED:
The "COPERNICUS/S2_SR_HARMONIZED" dataset in Earth Engine contains harmonized Sentinel-2 Level-2A (L2A) data. Level-2A data refers to top-of-atmosphere (TOA) reflectance data that has undergone atmospheric correction and radiometric calibration. This processing step aims to remove atmospheric effects and provide a consistent and comparable reflectance product. The harmonization process takes into account various factors like atmospheric conditions, sun angle, and surface properties.